## Layout Planning

In [1]:
%matplotlib inline
import pandas as pd
import numpy as np

In [2]:
data = pd.read_excel('120s PDM Data.xlsx',sheet_name = None)

### 讀進flow

In [3]:
xls = pd.ExcelFile('120s PDM Data.xlsx')
xls.sheet_names 
flow = data.get('Flow')
flow.head()

,TRAVELER,TRAV DESC,STEP NAME,RECIPE NAME,WS,AREA,RPT MINS,WS LOAD PCT,DISPATCH RPT,SUBSET,...,SUB-FAMILY,PROCESS,DENSITY,RPT VALIDATION STATUS,MODELED UTIL,AVG LOADSIZE,MODELED AVAIL,LOST TIME,RPT BASIS,MODELED CHAMBERS
0,V00H,100S PC,0000-00 START LOT START,PC LOT START,11-3START_WAFERS,F11 GENERAL,0.43,100.0,NaN,_,...,DDR III,D100S,4 GIG,NaN,1.00,25.0,1.00,0.0,NaN,1.0
1,V00H,100S PC,5450-21 STI OXIDE BACKSIDE WET ETCH,WET_100S_SWC_DHF100,11-3DNS_SU31_DHF_4,F11 WET PROCESS,0.70,0.0,NaN,_,...,DDR III,D100S,4 GIG,NaN,0.88,1.0,0.96,0.0,NaN,1.0
2,V00H,100S PC,5450-21 STI OXIDE BACKSIDE WET ETCH,WET_100S_SWC_DHF100,11-3DNS_SU3_DHF100,F11 WET PROCESS,0.70,5.0,NaN,_,...,DDR III,D100S,4 GIG,NaN,0.88,1.0,0.96,0.0,NaN,1.0
3,V00H,100S PC,5450-21 STI OXIDE BACKSIDE WET ETCH,WET_100S_SWC_DHF100,11-3DNS_SU31_DHF100,F11 WET PROCESS,0.35,95.0,NaN,_,...,DDR III,D100S,4 GIG,NaN,0.88,1.0,0.96,0.0,NaN,1.0
4,V00H,100S PC,5200-53 GATE NITRIDE DEP 2 CMP,CMP_100S_BUFF,11-3EBRA_FRXS_BUFF,F11 CMP,1.12,100.0,NaN,_,...,DDR III,D100S,4 GIG,NaN,0.75,1.0,0.86,0.0,1.0,1.0


In [4]:
flow = flow[flow['TRAV DESC'] == '120S PC']
flow.shape[0]

6326

In [5]:
#flow_dup = flow.drop_duplicates(subset = ['WS'],keep = 'first',inplace=False)
#flow_dup.head(400)
#flow_dup = flow_dup.(flow_dup[(flow_dup['AREA'].isin(['F11 GENERAL']))])
#flow_dup.head()

### 索引全部經過Workstation名稱

In [6]:
flow = flow[flow.AREA != 'F11 GENERAL']
flow.head()
step = flow['WS'][:-1].tolist()
print(step)


['11-3EBRA_FRXS_BUFF1', '11-3TEL_303I_PADOX', '11-3KLAT_FX200_THK', '11-3KLAT_F5X_THK', '11-3KLAT_FX100_THK', '11-3KLAT_F5X_THK_1', '11-3KLAT_FX200_THK_1', '11-3DNS_SS32_SCRB_DS', '11-3DNS_SS33_SCRB_DS', '11-3TEL_NS3S_SCRB_DS', '11-3TEL_303I_TEOS', '11-3HIKE_QULN_TEOS', '11-3KLAT_FX100_THK', '11-3KLAT_F5X_THK', '11-3KLAT_FX200_THK', '11-3KLAT_FX200_THK_1', '11-3KLAT_F5X_THK_1', '11-3DNS_SU33_DHF_APM', '11-3DNS_SU32_APM', '11-3DNS_SU32_CHX_6', '11-3DNS_SU32_49HF', '11-3DNS_SU32_DHF_SPM', '11-3NVLS_VEXT_AHMDLC', '11-3LAM_CORV2_O2_1_3', '11-3LAM_CORE4_O2_1_4', '11-3KLAT_CIRCL_MACR', '11-3AMAT_PRGT3_ARLHT', '11-3AMAT_PRSE_ARLHT', '11-3AMAT_PRGTP_ARLHT', '11-3KLAT_SS10K_OCD', '11-3DNS_SU32_CHX_6', '11-3DNS_SU32_APM', '11-3DNS_SU33_DHF_APM', '11-3TEL_CELI_100HF', '11-3NVLS_VEXT_AHMDLC', '11-3NVLS_VEXT_AHM', '11-3LAM_CORV2_O2_1_3', '11-3LAM_CORE4_O2_1_4', '11-3AMAT_PRSE_DARC', '11-3AMAT_PRGT3_ARLON', '11-3AMAT_PRGTP_ARLON', '11-3AMAT_PRGT3_DARC', '11-3DNS_SS32_SCRB_DS', '11-3DNS_SS33_SCRB_DS'

### 獲取WS與Equipment ID對應關係

In [7]:
Qty = data.get('Qty')

In [8]:
WS_EQ = pd.concat([Qty['WS NAME'],Qty['EQUIP ID']],axis=1)
WS_EQ.head() #機台對應 WS
WS_NAME = WS_EQ['WS NAME'].tolist()
#print(WS_NAME)

In [9]:
WS_EQ.head()

,WS NAME,EQUIP ID
0,11-3ABON_ZFB_N2P,AN2PB80101
1,11-3ABON_ZFB_N2P,AN2PB80102
2,11-3ABON_ZFB_N2P,AN2PB80103
3,11-3ABON_ZFB_N2P,AN2PB80104
4,11-3ABON_ZFB_N2P,AN2PB80105


### 投片數

In [10]:
wafer = input('Please enter the quantity of wafers:')
wafer_m = []
for i in step:
    if i in WS_NAME:
        w = round(int(wafer)/WS_EQ.groupby(['WS NAME']).size()[i],2)
        wafer_m.append(w)
    else:
        wafer_m.append(0)
    #print(i)
print ('Number of wafers in each machine:',wafer_m)

'''for i in step:
    if i not in WS_NAME:
        step.remove(i)
    else:
        None
print(step)'''

Please enter the quantity of wafers:5000
Number of wafers in each machine: [2500.0, 1250.0, 833.33, 1000.0, 1666.67, 833.33, 714.29, 1000.0, 5000.0, 625.0, 454.55, 2500.0, 1666.67, 1000.0, 833.33, 714.29, 833.33, 0, 0, 2500.0, 5000.0, 357.14, 178.57, 1666.67, 625.0, 2500.0, 333.33, 833.33, 833.33, 1666.67, 2500.0, 0, 0, 2500.0, 178.57, 384.62, 1666.67, 625.0, 185.19, 333.33, 416.67, 238.1, 1000.0, 5000.0, 625.0, 178.57, 384.62, 1666.67, 625.0, 1666.67, 1666.67, 416.67, 333.33, 2500.0, 555.56, 1000.0, 1666.67, 714.29, 0, 714.29, 714.29, 833.33, 208.33, 625.0, 1666.67, 1000.0, 833.33, 714.29, 714.29, 0, 714.29, 108.7, 0, 0, 2500.0, 0, 5000.0, 357.14, 1000.0, 1666.67, 714.29, 0, 714.29, 714.29, 833.33, 0, 1666.67, 625.0, 208.33, 833.33, 625.0, 108.7, 0, 833.33, 1666.67, 0, 1666.67, 1000.0, 833.33, 714.29, 714.29, 0, 714.29, 1666.67, 5000.0, 5000.0, 5000.0, 1000.0, 2500.0, 1666.67, 1250.0, 1000.0, 1666.67, 833.33, 833.33, 1666.67, 1000.0, 714.29, 0, 714.29, 714.29, 833.33, 108.7, 0, 1666.6

'for i in step:\n    if i not in WS_NAME:\n        step.remove(i)\n    else:\n        None\nprint(step)'

In [11]:
print(len(step))
print(len(wafer_m))

6108
6108


### 找出WS有多少台機台，並平均分配wafer數量

In [12]:
col = ['3','4','7','8','9','C','D','E']
#print(wafer_in_each_machine)
wafer_in_each_machine = pd.DataFrame(columns = col)
machine_num = pd.DataFrame(columns = col)
for i in step:
    f1 = WS_EQ[(WS_EQ['WS NAME'] == i) & (WS_EQ['EQUIP ID'].str.get(5) == '3')].shape[0]
    f2 = WS_EQ[(WS_EQ['WS NAME'] == i) & (WS_EQ['EQUIP ID'].str.get(5) == '4')].shape[0]
    f3 = WS_EQ[(WS_EQ['WS NAME'] == i) & (WS_EQ['EQUIP ID'].str.get(5) == '7')].shape[0]
    f4 = WS_EQ[(WS_EQ['WS NAME'] == i) & (WS_EQ['EQUIP ID'].str.get(5) == '8')].shape[0]
    f5 = WS_EQ[(WS_EQ['WS NAME'] == i) & (WS_EQ['EQUIP ID'].str.get(5) == '9')].shape[0]
    f6 = WS_EQ[(WS_EQ['WS NAME'] == i) & (WS_EQ['EQUIP ID'].str.get(5) == 'C')].shape[0]
    f7 = WS_EQ[(WS_EQ['WS NAME'] == i) & (WS_EQ['EQUIP ID'].str.get(5) == 'D')].shape[0]
    f8 = WS_EQ[(WS_EQ['WS NAME'] == i) & (WS_EQ['EQUIP ID'].str.get(5) == 'E')].shape[0]
    machine = np.array([f1,f2,f3,f4,f5,f6,f7,f8])
    machine_D = pd.DataFrame([machine],columns = col)
    machine_num = machine_num.append(machine_D)
    for j in range(8):
        if i in WS_NAME:
            num = wafer_m[step.index(i)]*machine[j]
        else:
            num = 0
        machine[j] = num
    machine = pd.DataFrame([machine],columns = col)
    #print(machine)
    wafer_in_each_machine = wafer_in_each_machine.append(machine)

In [13]:
machine_num.head()

,3,4,7,8,9,C,D,E
0,0,0,0,0,0,2,0,0
0,0,0,1,0,0,3,0,0
0,3,0,0,0,0,3,0,0
0,2,0,2,0,0,1,0,0
0,0,0,3,0,0,0,0,0


In [14]:
wafer_in_each_machine['WS NAME'] = step 
wafer_in_each_machine.set_index('WS NAME',inplace = True)
wafer_in_each_machine.head()

,3,4,7,8,9,C,D,E
WS NAME,,,,,,,,
11-3EBRA_FRXS_BUFF1,0,0,0,0,0,5000,0,0
11-3TEL_303I_PADOX,0,0,1250,0,0,3750,0,0
11-3KLAT_FX200_THK,2499,0,0,0,0,2499,0,0
11-3KLAT_F5X_THK,2000,0,2000,0,0,1000,0,0
11-3KLAT_FX100_THK,0,0,5000,0,0,0,0,0


In [15]:
wafer_in_each_machine.shape
w = wafer_in_each_machine.values
#print(w)
w.shape

(6108, 8)

### 照比例分配搬運量，assign weight

In [16]:
a = range(8)
m = []
for k in range(len(step)-1):
    for j in a:
        for i in a:
            n = max(w[k,j]-w[k+1,j],0)*w[k+1,a]//(int(wafer)-w[k+1,j]+1) #避免分母為零
        m.append(n)      
step_m = np.array(m)
#print(step_m)
#print(step_m.shape)
carry = np.vsplit(step_m, len(step)-1)
for i in range(len(step)-1):
    for j in range(8):
        carry[i][j][j] = min(w[i,j],w[i+1,j])
        carry = np.array(carry)
#print(carry)

In [31]:
def weight(distance,carry,step):
    total_carry = []
    for i in range(len(step)-1):
        carry_weight = np.multiply(carry[i],distance)
        total_carry_step = np.sum(carry_weight)-np.trace(carry_weight) 
        total_carry.append(total_carry_step)
        print (i,'-',i+1)
        print (carry_weight)
        print ('Step carriage:',total_carry_step)
        print ('-----------------------------------------')
    print ('Total carriage:',sum(total_carry))
    return 
weight(distance = np.ones((8,8)),carry = carry,step = step)

0 - 1
[[0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 1249.0 0.0 0.0 3750.0 0.0 0.0]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]]
Step carriage: 1249.0
-----------------------------------------
1 - 2
[[0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [624.0 0.0 0.0 0.0 0.0 624.0 0.0 0.0]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [1249.0 0.0 0.0 0.0 0.0 2499.0 0.0 0.0]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]]
Step carriage: 2497.0
-----------------------------------------
2 - 3
[[2000.0 0.0 332.0 0.0 0.0 166.0 0.0 0.0]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [749.0 0.0 749.0 0.0 0.0 1000.0 0.0 0.0]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0

-----------------------------------------
220 - 221
[[0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [555.0 0.0 0.0 0.0 555.0 555.0 0.0 0.0]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [1110.0 0.0 0.0 0.0 1110.0 1110.0 0.0 0.0]]
Step carriage: 4995.0
-----------------------------------------
221 - 222
[[0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]]
Step carriage: 0.0
-----------------------------------------
222 - 223
[[0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.

Step carriage: 0.0
-----------------------------------------
373 - 374
[[0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]]
Step carriage: 0.0
-----------------------------------------
374 - 375
[[0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]]
Step carriage: 0.0
-----------------------------------------
375 - 376
[[0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0

Step carriage: 0.0
-----------------------------------------
511 - 512
[[0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]]
Step carriage: 0.0
-----------------------------------------
512 - 513
[[0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]]
Step carriage: 0.0
-----------------------------------------
513 - 514
[[0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0

Step carriage: 0.0
-----------------------------------------
637 - 638
[[0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]]
Step carriage: 0.0
-----------------------------------------
638 - 639
[[0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]]
Step carriage: 0.0
-----------------------------------------
639 - 640
[[0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0

...

Step carriage: 0.0
-----------------------------------------
5564 - 5565
[[0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]]
Step carriage: 0.0
-----------------------------------------
5565 - 5566
[[0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]]
Step carriage: 0.0
-----------------------------------------
5566 - 5567
[[0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0

-----------------------------------------
5697 - 5698
[[0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]]
Step carriage: 0.0
-----------------------------------------
5698 - 5699
[[0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]]
Step carriage: 0.0
-----------------------------------------
5699 - 5700
[[0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.

 [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]]
Step carriage: 0.0
-----------------------------------------
5820 - 5821
[[0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]]
Step carriage: 0.0
-----------------------------------------
5821 - 5822
[[0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]]
Step carriage: 0.0
-----------------------------------------
5822 - 5823
[[0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.

Step carriage: 0.0
-----------------------------------------
5940 - 5941
[[0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]]
Step carriage: 0.0
-----------------------------------------
5941 - 5942
[[0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]]
Step carriage: 0.0
-----------------------------------------
5942 - 5943
[[0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0

6076 - 6077
[[0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]]
Step carriage: 0.0
-----------------------------------------
6077 - 6078
[[0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]]
Step carriage: 0.0
-----------------------------------------
6078 - 6079
[[0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]]

In [32]:
'''carry = carry.reshape(len(step)-1,64)
print(carry)
print(carry.shape)'''
#import itertools
'''fab = []
test_data = ['3','4','7','8','9','C','D','E']
for i in itertools.combinations_with_replacement(test_data, 2):
    fab.append(i)
print(fab)'''

"fab = []\ntest_data = ['3','4','7','8','9','C','D','E']\nfor i in itertools.combinations_with_replacement(test_data, 2):\n    fab.append(i)\nprint(fab)"

## Add one more tool, find the optimal layout strategy

In [33]:
change_step = input('Please enter the changed step (start from zero): ') #修改機台數的步驟
add_machine = input('Please enter the quantity of equipment added: ')
change_data = wafer_in_each_machine.iloc[int(change_step)-1:int(change_step)+2,] #提取該步驟
change_WS = step[int(change_step)-1:int(change_step)+2] #提取索引
#print(change_step)
#print(change_data)
#print(change_WS)

Please enter the changed step (start from zero): 600
Please enter the quantity of equipment added: 3


In [34]:
data = change_data.values 
data[1,1] 

0

In [35]:
data = change_data.values #提取部分值 array
for l in range(8):
    data[1,l] = data[1,l]+int(add_machine)
    a = range(8)
    c_m = []
    for k in range(len(change_WS)-1):
        for j in a:
            for i in a:
                c_n = max(data[k,j]-data[k+1,j],0)*data[k+1,a]//(int(wafer)-data[k+1,j]+1) #避免分母為零
            c_m.append(c_n)      
    c_step_m = np.array(c_m)
    #print(c_step_m)
    #print(c_step_m.shape)
    c_carry = np.vsplit(c_step_m, len(change_data)-1)
    for i in range(len(change_data)-1):
        for j in range(8):
            c_carry[i][j][j] = min(data[i,j],data[i+1,j])
            c_carry = np.array(c_carry)
    #print(c_carry)
    #print('---------------------')
    print (col[l])
    weight(distance = np.ones((8,8)),carry = c_carry,step = change_WS)
    print('==========================================')
    print('                                           ')
    data[1,l] = data[1,l]+int(add_machine)

3
0 - 1
[[0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [627.0 0.0 2499.0 0.0 0.0 1875.0 0.0 0.0]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]]
Step carriage: 3126.0
-----------------------------------------
1 - 2
[[628.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [297.0 0.0 714.0 0.0 222.0 446.0 446.0 371.0]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [146.0 0.0 146.0 0.0 109.0 1071.0 219.0 182.0]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]]
Step carriage: 2584.0
-----------------------------------------
Total carriage: 5710.0
                                           
4
0 - 1
[[0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [630.0 2.0

In [ ]:
#可直接讀近距離矩陣
'''
distance = pd.read_csv('')
distance = distance.values'''